# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

In [57]:
import pandas as pd
import numpy as np

In [58]:
data = pd.read_csv('../data/trip_duration_task.csv')
data = data.drop_duplicates()
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,id1080784,2.0,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,400
1,id0889885,1.0,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,1100
2,id0857912,2.0,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,1635
3,id3744273,2.0,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,1141
4,id0232939,1.0,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,848


In [59]:
import time
import datetime
#s ="2016-02-29 16:47:01"
def t(s):
    return time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple())
data['dropoff_datetime']=data['dropoff_datetime'].map(t)
data['pickup_datetime']=data['pickup_datetime'].map(t)

In [3]:
def d(x):
    return int(x[a:b])
#a=0
#b=4
#data['pickup_god'] = data['pickup_datetime'].map(d)
#data['dropoff_god'] = data['dropoff_datetime'].map(d)
#a=5
#b=7
#data['pickup_mes'] = data['pickup_datetime'].map(d)
#data['dropoff_mes'] = data['dropoff_datetime'].map(d)
a=8
b=10
data['pickup_day'] = data['pickup_datetime'].map(d)
data['dropoff_day'] = data['dropoff_datetime'].map(d)
a=11
b=13
data['pickup_chas'] = data['pickup_datetime'].map(d)
data['dropoff_chas'] = data['dropoff_datetime'].map(d)
a=14
b=16
data['pickup_min'] = data['pickup_datetime'].map(d)
data['dropoff_min'] = data['dropoff_datetime'].map(d)
a=17
b=19
data['pickup_sec'] = data['pickup_datetime'].map(d)
data['dropoff_sec'] = data['dropoff_datetime'].map(d)
data.drop(['pickup_datetime', 'dropoff_datetime'], axis=1, inplace=True)
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,pickup_day,dropoff_day,pickup_chas,dropoff_chas,pickup_min,dropoff_min,pickup_sec,dropoff_sec
0,id1080784,2.0,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,400,29,29,16,16,40,47,21,1
1,id0889885,1.0,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,1100,11,11,23,23,35,53,37,57
2,id0857912,2.0,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,1635,21,21,17,18,59,26,33,48
3,id3744273,2.0,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,1141,5,5,9,10,44,3,31,32
4,id0232939,1.0,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,848,17,17,6,6,42,56,23,31


In [60]:
data.drop(['id'], axis=1, inplace=True)
data.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2.0,1.456742e+09,1.456743e+09,1,-73.953918,40.778873,-73.963875,40.771164,400
1,1.0,1.457718e+09,1.457719e+09,2,-73.988312,40.731743,-73.994751,40.694931,1100
2,2.0,1.456056e+09,1.456058e+09,2,-73.997314,40.721458,-73.948029,40.774918,1635
3,2.0,1.451965e+09,1.451967e+09,6,-73.961670,40.759720,-73.956779,40.780628,1141
4,1.0,1.455670e+09,1.455671e+09,1,-74.017120,40.708469,-73.988182,40.740631,848


In [61]:
for st in data.columns:
    if type(st[1]) == int:
        a=np.median(st)
    elif type(st[1]) == float:
        print(1);
        a=np.average(st)
    else:
        a=data.mode(axis= 0, numeric_only= False, dropna = True)[st][0]
def f(x):
    if pd.isna(x):
        return a
    else:
        return x
for st in data.columns:
    data[st]=data[st].map(f)

In [62]:
data.to_csv('../data/trip_duration_task_obr.csv')

In [63]:
data = pd.read_csv('../data/trip_duration_task_obr.csv').sample(20000).reset_index(drop=True)
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2.0,1.453284e+09,1.453285e+09,3,-73.978508,40.782879,-73.978409,40.751518,917
1,2.0,1.460542e+09,1.460543e+09,1,-73.976028,40.748531,-73.969757,40.761204,865
2,2.0,1.466607e+09,1.466607e+09,6,-73.998611,40.740551,-74.006638,40.743999,123
3,2.0,1.461575e+09,1.461575e+09,1,-74.005547,40.739964,-73.997215,40.722374,780
4,1.0,1.461090e+09,1.461091e+09,1,-73.977631,40.758034,-73.977608,40.749466,840


In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   vendor_id          20000 non-null  float64
 1   pickup_datetime    20000 non-null  float64
 2   dropoff_datetime   20000 non-null  float64
 3   passenger_count    20000 non-null  int64  
 4   pickup_longitude   20000 non-null  float64
 5   pickup_latitude    20000 non-null  float64
 6   dropoff_longitude  20000 non-null  float64
 7   dropoff_latitude   20000 non-null  float64
 8   trip_duration      20000 non-null  int64  
dtypes: float64(7), int64(2)
memory usage: 1.4 MB


In [65]:
data.loc[ : , data.columns != 'trip_duration'].corrwith(data['trip_duration'])

vendor_id           -0.002827
pickup_datetime      0.006489
dropoff_datetime     0.007226
passenger_count      0.023321
pickup_longitude     0.047193
pickup_latitude     -0.000470
dropoff_longitude    0.054456
dropoff_latitude    -0.028673
dtype: float64

In [66]:
from sklearn.preprocessing import scale
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression

In [67]:
#data_shuffled = shuffle(data, random_state=123)
#X = scale(data_shuffled[data_shuffled.loc[ : , data.columns != 'trip_duration']])
#y = data_shuffled["trip_duration"]

In [68]:
y = data["trip_duration"]
X = data.drop(["trip_duration"], axis=1)

In [69]:
y

0         917
1         865
2         123
3         780
4         840
         ... 
19995    1216
19996     407
19997    2080
19998    1101
19999     740
Name: trip_duration, Length: 20000, dtype: int64

In [70]:
X

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2.0,1.453284e+09,1.453285e+09,3,-73.978508,40.782879,-73.978409,40.751518
1,2.0,1.460542e+09,1.460543e+09,1,-73.976028,40.748531,-73.969757,40.761204
2,2.0,1.466607e+09,1.466607e+09,6,-73.998611,40.740551,-74.006638,40.743999
3,2.0,1.461575e+09,1.461575e+09,1,-74.005547,40.739964,-73.997215,40.722374
4,1.0,1.461090e+09,1.461091e+09,1,-73.977631,40.758034,-73.977608,40.749466
...,...,...,...,...,...,...,...,...
19995,1.0,1.451915e+09,1.451917e+09,1,-73.958488,40.713387,-73.986519,40.762260
19996,2.0,1.462935e+09,1.462936e+09,1,-73.975143,40.755959,-73.969559,40.764851
19997,2.0,1.458757e+09,1.458759e+09,1,-73.958679,40.721180,-73.947250,40.823238
19998,1.0,1.455489e+09,1.455490e+09,1,-73.957558,40.709084,-73.991394,40.749249


In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [72]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((16000, 8), (16000,), (4000, 8), (4000,))

In [73]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [74]:
lr = LinearRegression().fit(X_train, y_train)

In [75]:
lr.predict(X_test)

array([ 759.00000003,  654.00000003,  142.00000003, ...,  220.00000003,
       1015.00000003,  886.00000003])

In [76]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

In [77]:
y_pred = lr.predict(X_test)

In [78]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lr.score(X_test, y_test)}')

MAE: 3.4332249751245494e-08
MSE: 1.1787034504616954e-15
RMSE: 3.4332250879627675e-08
MAPE: 9.543962701012525e-11
R^2: 1.0


In [79]:
len(lr.coef_)
l=zip(data.columns, lr.coef_)
print(list(l))

[('vendor_id', -4.494765401565355e-12), ('pickup_datetime', -0.9999999999999989), ('dropoff_datetime', 0.9999999999999987), ('passenger_count', -5.4874562863970476e-12), ('pickup_longitude', 1.1997924768134202e-10), ('pickup_latitude', -1.1985716982220585e-13), ('dropoff_longitude', -4.383357422173528e-12), ('dropoff_latitude', -4.0673795623123505e-11)]


In [80]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np

In [81]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

In [82]:
lasso_optimal = GridSearchCV(Lasso(), parameters).fit(X_train, y_train)
alpl=lasso_optimal.best_params_['alpha']
print(alpl)

C:\machine_learning\venv\lib\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.353e+10, tolerance: 1.280e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = 

C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.577e+10, tolerance: 1.327e+07
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.357e+10, tolerance: 1.280e+07
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.932e+10, toleranc

C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.192e+10, tolerance: 1.448e+07
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.584e+10, tolerance: 1.327e+07
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.363e+10, toleranc

0.0


In [83]:
#L1
lasso = Lasso(alpha=alpl).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

C:\Users\DNS\AppData\Local\Temp\ipykernel_15056\1307015766.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso = Lasso(alpha=alpl).fit(X_train, y_train)
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


MAE: 567.7645102689846
MSE: 9409316.072021706
RMSE: 3067.4608509354616
MAPE: 2.243812253229873
R^2: 0.005753258044324583


C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.754e+10, tolerance: 1.764e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


array([-5.65945831e-01, -5.54290691e-04,  5.57842651e-04,  5.97375150e+01,
        2.80203737e+03, -6.77813112e-01,  4.55361492e+03, -3.61104783e+03])

In [84]:
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)
alpr=ridge_optimal.best_params_['alpha']
print(alpr)

C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.42859e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.38102e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.4612e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.45287e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216

C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.59525e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.58757e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.58121e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.58056e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:21

0.0


C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.61957e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.43467e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [85]:
#L2
ridge = Ridge(alpha=alpr).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.43467e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


MAE: 1.0494042052856534e-06
MSE: 3.113564889034071e-11
RMSE: 5.579932695861188e-06
MAPE: 4.154167671302484e-09
R^2: 1.0


array([-1.02952108e-09, -9.99999998e-01,  9.99999998e-01,  1.08640236e-07,
        5.09532924e-06, -1.23123755e-09,  8.28220898e-06, -6.56767466e-06])

In [86]:
#Poli
from sklearn.preprocessing import PolynomialFeatures
#y = y.values.reshape(-1,1)
#X = X.values.reshape(-1,1)
p = PolynomialFeatures(2)
X_p=p.fit_transform(X_train) 
X_p


array([[ 1.00000000e+00,  1.00000000e+00,  1.45870076e+09, ...,
         5.47294269e+03, -3.01487087e+03,  1.66079693e+03],
       [ 1.00000000e+00,  2.00000000e+00,  1.45809172e+09, ...,
         5.46208981e+03, -3.01192299e+03,  1.66084419e+03],
       [ 1.00000000e+00,  2.00000000e+00,  1.45967494e+09, ...,
         5.47199564e+03, -3.01497488e+03,  1.66119897e+03],
       ...,
       [ 1.00000000e+00,  1.00000000e+00,  1.45624162e+09, ...,
         5.47467107e+03, -3.01478323e+03,  1.66017608e+03],
       [ 1.00000000e+00,  2.00000000e+00,  1.45863162e+09, ...,
         5.47420028e+03, -3.01434286e+03,  1.65983384e+03],
       [ 1.00000000e+00,  1.00000000e+00,  1.45308639e+09, ...,
         5.45597705e+03, -3.01147244e+03,  1.66220756e+03]])

In [89]:
lr2 = LinearRegression().fit(X_p, y_train)
X_p_test=p.fit_transform(X_test)
y_pred = lr2.predict(X_p_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lr2.score(X_p_test, y_test)}')
lr2.coef_

MAE: 0.25342350832674104
MSE: 0.6169304255252123
RMSE: 0.785449187105832
MAPE: 0.002082598586399572
R^2: 0.9999999348113018


array([ 2.20085833e-06, -2.04104278e-04, -1.03105921e-04,  1.05395473e-04,
       -2.67498464e-08,  7.42986776e-10, -5.32529817e-06, -5.28131657e-10,
       -6.72970413e-10,  1.81352961e-05, -6.20258073e-07,  6.20285388e-07,
       -7.18022881e-05,  1.48914798e-03,  4.99833063e-03,  1.47392797e-03,
       -7.52022843e-04,  1.32578691e-09, -2.67153823e-09,  1.46666427e-05,
        3.61096773e-03, -1.79567475e-06,  8.66240627e-03, -1.54047649e-03,
        1.34575100e-09, -1.46666150e-05, -3.61096125e-03,  1.79653514e-06,
       -8.66239628e-03,  1.54047190e-03, -2.09480756e-04,  2.30397114e-06,
       -7.05121323e-04,  2.45406504e-06, -1.22596281e-06, -1.55674423e-06,
        3.74071076e-04, -1.09005461e-07,  7.71602941e-08, -3.08525520e-03,
        3.80981692e-04, -2.04823399e-04,  1.51580187e-08, -8.13251576e-08,
       -1.71357193e-07])

In [90]:
#Poli1
parameters = {'alpha': np.arange(0, 1, 0.1)}
lasso_p_optimal = GridSearchCV(Lasso(), parameters).fit(X_p, y_train)
alpl_p=lasso_p_optimal.best_params_['alpha']
print(alpl_p)


C:\machine_learning\venv\lib\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.339e+10, tolerance: 1.280e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = 

C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.561e+10, tolerance: 1.327e+07
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.339e+10, tolerance: 1.280e+07
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.912e+10, toleranc

C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.166e+10, tolerance: 1.448e+07
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.561e+10, tolerance: 1.327e+07
  model = cd_fast.enet_coordinate_descent(
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.340e+10, toleranc

0.0


C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.734e+10, tolerance: 1.764e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


In [91]:
lasso_p = Lasso(alpha = alpl_p).fit(X_p, y_train)
y_pred = lasso_p.predict(X_p_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lasso_p.score(X_p_test, y_test)}')
lasso_p.coef_

C:\Users\DNS\AppData\Local\Temp\ipykernel_15056\2865582534.py:1: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso_p = Lasso(alpha = alpl_p).fit(X_p, y_train)
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


MAE: 567.9221808396056
MSE: 9389297.565439297
RMSE: 3064.1960716375993
MAPE: 2.2238680809370126
R^2: 0.007868537709289858


C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.734e+10, tolerance: 1.764e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


array([ 0.00000000e+00, -8.27082167e-01, -5.52236612e-04,  5.57440841e-04,
        6.95419036e+01,  7.05098640e+03,  3.39758500e-01,  4.72722577e+03,
       -8.50131148e+03, -3.00319790e-01,  7.15855077e-08,  3.04791563e-11,
       -4.24178829e-01, -1.25238247e-02, -4.17440305e-02, -2.25836472e-02,
        7.38278790e-03, -1.92623418e-13,  9.54307315e-14,  1.18787583e-07,
        3.66810325e-06,  3.36692039e-10, -7.53510440e-09, -1.35370900e-07,
        1.89995704e-13, -1.75331399e-09, -9.28648210e-08, -5.06970732e-11,
       -1.60582237e-08, -1.14406269e-07, -1.78732732e+01,  6.98983333e-01,
       -1.09948728e-01,  4.54619208e-01,  2.06371782e-01,  1.57855755e+02,
        8.67719278e-03, -1.66898224e+02,  3.76023932e+01,  2.12264889e-02,
        1.07893787e-01, -1.94841980e-02,  7.87024816e+01, -2.14424820e+01,
        7.93589425e+01])

In [92]:
#Poli2
ridge_p_optimal = GridSearchCV(Ridge(), parameters).fit(X_p, y_train)
alpr_p=ridge_p_optimal.best_params_['alpha']
print(alpr_p)

C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.5403e-38): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.53699e-38): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.55111e-38): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.08245e-38): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216

0.7000000000000001


In [93]:
ridge_p = Ridge(alpha = alpr_p).fit(X_p, y_train)
y_pred = ridge_p.predict(X_p_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {ridge_p.score(X_p_test, y_test)}')
ridge_p.coef_

MAE: 0.6046773854304948
MSE: 0.9614461631476595
RMSE: 0.9805336114318873
MAPE: 0.0025024675292102754
R^2: 0.9999998984076305


array([-3.59200567e-04, -1.78649681e-01, -3.14474182e-01,  3.14474345e-01,
       -9.58103929e-04,  2.07354961e-06, -2.46267021e-03, -1.79110093e-06,
       -6.02590027e-07, -1.63017724e-04, -1.54090339e-06,  1.54083376e-06,
        3.21764340e-04, -1.30101953e-02, -1.06644241e-02,  1.83605626e-02,
        2.86291541e-02,  2.65988501e-09, -5.33426168e-09,  1.02608415e-05,
        2.19555508e-03,  1.06318729e-05,  6.36550458e-03, -7.69797148e-04,
        2.67437657e-09, -1.02506224e-05, -2.19555490e-03, -1.06276649e-05,
       -6.36550022e-03,  7.69803441e-04, -4.07883392e-01,  7.37015256e-02,
        1.51940471e-03,  7.30714374e-02, -3.99121758e-02, -3.08490127e-03,
        1.08196335e-01,  1.60378386e-04,  1.05215299e-04,  4.80162640e-02,
        1.16587829e-01, -2.00136940e-01,  3.75190617e-04, -1.18554710e-04,
       -1.64510832e-04])

In [94]:
lasso = Lasso(alpha=alpl).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

C:\Users\DNS\AppData\Local\Temp\ipykernel_15056\1419376383.py:1: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso = Lasso(alpha=alpl).fit(X_train, y_train)
C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


MAE: 567.7645102689846
MSE: 9409316.072021706
RMSE: 3067.4608509354616
MAPE: 2.243812253229873
R^2: 0.005753258044324583


C:\machine_learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.754e+10, tolerance: 1.764e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


array([-5.65945831e-01, -5.54290691e-04,  5.57842651e-04,  5.97375150e+01,
        2.80203737e+03, -6.77813112e-01,  4.55361492e+03, -3.61104783e+03])

In [95]:
class MyRidge:
    def __init__( self, learning_rate=0.0001, iterations=1000, alpha=0.5 ) :
        self.learning_rate = learning_rate        
        self.iterations = iterations        
        self.alpha = alpha

    def transform_(self, x):
        return np.concatenate((np.ones((len(x), 1)), x), axis = 1)

    def loss_func(self, x, y, w):
        return sum(y - np.dot(x, w)) ** 2  + self.alpha * (np.matmul(w, w))/x.shape[0]

    def fit(self, x, y):
        dist = np.inf
        eps = 1e-20
        X = self.transform_(x)

        w = np.zeros(X.shape[1])
        iter = 0

        while iter <= self.iterations:
            loss = self.loss_func(X, y, w)
            w = w - ((self.learning_rate * 2 * np.dot(X.T, np.dot(X, w) - y)) + ( 2 * self.alpha * w ))/X.shape[0]
            dist = np.abs(loss - self.loss_func(X, y, w))
            iter += 1
            
            if(dist <= eps):
                break
        self.w = w
        return self
        

    def predict(self, x):
        return np.dot(self.transform_(x), self.w)

In [96]:
def R2(y_test, y_pred):
    numerator = ((y_test - y_pred) ** 2).sum()
    denominator = (
        (y_test - np.average(y_test, axis=0)) ** 2
        ).sum()
    return 1 - (numerator / denominator)
    
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred)/y_test))
    
def RMSE(y_test, y_pred):
    return np.sqrt(MSE(y_test, y_pred))
    
def MSE(y_test, y_pred):
    return np.mean((y_test - y_pred)**2)
    
def MAE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred)))

In [97]:
ridge = MyRidge()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {MAE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MAPE: {MAPE(y_test, y_pred)}')
print(f'R^2: {R2(y_test, y_pred)}')


OverflowError: (34, 'Result too large')

In [21]:
data = data.dropna()

In [6]:
data

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2.0,1.456742e+09,1.456743e+09,1,-73.953918,40.778873,-73.963875,40.771164,400
1,1.0,1.457718e+09,1.457719e+09,2,-73.988312,40.731743,-73.994751,40.694931,1100
2,2.0,1.456056e+09,1.456058e+09,2,-73.997314,40.721458,-73.948029,40.774918,1635
3,2.0,1.451965e+09,1.451967e+09,6,-73.961670,40.759720,-73.956779,40.780628,1141
4,1.0,1.455670e+09,1.455671e+09,1,-74.017120,40.708469,-73.988182,40.740631,848
...,...,...,...,...,...,...,...,...,...
729317,2.0,1.463816e+09,1.463816e+09,2,-73.965919,40.789780,-73.952637,40.789181,296
729318,1.0,1.456080e+09,1.456081e+09,1,-73.996666,40.737434,-74.001320,40.731911,315
729319,1.0,1.460725e+09,1.460726e+09,1,-73.997849,40.761696,-74.001488,40.741207,673
729320,1.0,1.466308e+09,1.466309e+09,1,-74.006706,40.708244,-74.013550,40.713814,447


In [125]:
from sklearn.model_selection import train_test_split
data=data.sample(20000)
X = data.iloc[:, :-1].values
Y = data.iloc[:, 8].values
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 0 )

In [135]:
class RidgeRegression_() :
      
    def __init__(self, learning_rate=0.01, iterations=1000, Alpha=0.0) :
          
        self.learning_rate = learning_rate        
        self.iterations = iterations        
        self.Alpha = Alpha
       
    def fit( self, X, Y ) :  
        self.m, self.n = X.shape
       
        self.W = np.zeros(self.n)
          
        self.b = 0        
        self.X = X        
        self.Y = Y
          
        for i in range( self.iterations ) :            
            self.update_weights()      
        return self
      
    def update_weights( self ) :           
        Y_pred = self.predict( self.X )
   
        dW = ( - ( 2 * ( self.X.T ).dot( self.Y - Y_pred ) ) +               
               ( 2 * self.Alpha * self.W ) ) / self.m     
        db = - 2 * np.sum( self.Y - Y_pred ) / self.m 
   
        self.W = self.W - self.learning_rate * dW    
        self.b = self.b - self.learning_rate * db        
        return self
      
    def predict( self, X ) :    
        return X.dot( self.W ) + self.b

In [103]:
class Ridge_:
        
    def __init__(self, Alpha=1.0):
        self.Alpha = Alpha;
    
    def fit(self, X, y):
        m, n = np.shape(X)
        upper_half = np.hstack((np.ones((m, 1)), X))
        lower = np.zeros((n, n))
        np.fill_diagonal(lower, np.sqrt(1.0))
        lower_half = np.hstack((np.zeros((n, 1)), lower))
        X = np.vstack((upper_half, lower_half))
        y = np.append(y, np.zeros(n))
        self.coef_ = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))
        return self
    
    def predict(self, X):
        return np.dot(X, self.coef_[1:9]) + self.coef_[0]
        
    def MAE_(self, y_test, y_pred):
        return sum(abs(y_test - y_pred))/y_test.shape[0]

    def MSE_(self, y_test, y_pred):
        return sum((y_test - y_pred)**2)/y_test.shape[0]

    def RMSE_(self, y_test, y_pred):
        return np.sqrt(sum((y_test - y_pred)**2)/y_test.shape[0])

    def MAPE_(self, y_test, y_pred):
        return (sum(abs((y_test - y_pred)/y_test)))/y_test.shape[0]

    def R_2(self, y_test, y_pred):
        return 1 - (sum((y_test - y_pred)**2)/y_test.shape[0])/(sum((y_test - np.mean(y_test))**2)/y_test.shape[0])

In [136]:
lr = RidgeRegression_(Alpha=1.0).fit(X_train, y_train)

C:\Users\DNS\AppData\Local\Temp\ipykernel_11420\1908087113.py:25: RuntimeWarning: overflow encountered in matmul
  dW = (self.X @ (A-self.Y).T)/self.m
C:\Users\DNS\AppData\Local\Temp\ipykernel_11420\1908087113.py:25: RuntimeWarning: invalid value encountered in matmul
  dW = (self.X @ (A-self.Y).T)/self.m
C:\Users\DNS\AppData\Local\Temp\ipykernel_11420\1908087113.py:32: RuntimeWarning: invalid value encountered in subtract
  self.W = self.W - self.learning_rate * dW


In [123]:
y_pred = lr.predict(X_test)

In [137]:
lr.predict(X_test)

array([nan, nan, nan, ..., nan, nan, nan])

In [107]:
y_test

array([1423,  321,  509, ...,  494,  301,  428], dtype=int64)

In [108]:
y_test.shape[0]

6000

In [109]:
lr.coef_

array([-2.65143630e+02, -4.40161123e-02, -1.00035251e+00,  1.00035251e+00,
       -3.72797363e-03, -1.43400516e+00,  1.29444602e+00, -9.22558963e-01,
        1.08743735e+00])

In [110]:
print('MAE: ', lr.MAE_(y_test, y_pred))
print('MSE: ', lr.MSE_(y_test, y_pred))
print('RMAE: ', lr.RMSE_(y_test, y_pred))
print('MApE: ', lr.MAPE_(y_test, y_pred))
print('R_2: ', lr.R_2(y_test, y_pred))

MAE:  0.21721491606905813
MSE:  1.7789825879178076
RMAE:  1.3337850606142685
MApE:  0.0005221186411883252
R_2:  0.9999998762860768


In [38]:
class LinReg:
    
    def Ridge(self, Alpha):
        self.Alpha = Alpha;
        
    def __init__(self, lr=0.001, iters=5000, eps=0.001, Alpha=0.0):
        self.lr = lr
        self.iters = iters
        self.eps = eps
        self.Alpha = Alpha;
    
    def fit(self, X, y):
        dist = np.inf
        w = np.zeros(X.shape[1])
        iter = 0
        while dist > self.eps and iter <= self.iters:
            loss = self.fun_error(X, y, w)
            w = w - (self.lr * 2 * np.dot(X.T, np.dot(X, w) - y) / X.shape[0]) + self.Alpha * 2 / X.shape[0]
            dist = np.abs(loss - self.fun_error(X, y, w))
            iter += 1
        self.w = w  
    
    def transform(self, X):
        return np.concatenate(np.ones((len(X), 1)), axis=1)


    def fun_error(self, X, y, w):
        return sum((y - np.dot(X, w)) ** 2) / X.shape[0]
    
    def predict(self, X):
        return np.dot(X, self.w)
        
    

    
    def MAE_(self, y_test, y_pred):
        return sum(abs(y_test - y_pred))/y_test.shape

    def MSE_(self, y_test, y_pred):
        return sum((y_test - y_pred)**2)/y_test.shape

    def RMSE_(self, y_test, y_pred):
        return np.sqrt(sum((y_test - y_pred)**2)/y_test.shape)

    def MAPE_(self, y_test, y_pred):
        return (sum(abs((y_test - y_pred)/y_test)))/y_test.shape

    def R_2(self, y_test, y_pred):
        return 1 - (MSE(y_test, y_pred))/(sum(y_test - np.mean(y_test))/y_test.shape)

In [213]:
lr = LinReg()
lr.fit(X, y)
y_pred = lr.predict(X_test)
print(y_pred.shape, y_test.shape)
print(y_pred)
print(y_test)
print('MAE: ', lr.MAE_(y_test, y_pred))
print('MSE: ', lr.MSE_(y_test, y_pred))
print('RMAE: ', lr.RMSE_(y_test, y_pred))
print('MApE: ', lr.MAPE_(y_test, y_pred))
print('R_2: ', lr.R_2(y_test, y_pred))

(4000,) (4000,)
[1.63002313e+178 1.61513801e+178 1.62312942e+178 ... 1.62499022e+178
 1.61870346e+178 1.62749438e+178]
17356    543
5296     803
4018     274
5003     334
14442    502
        ... 
15424    688
15397    166
14906    327
9719     595
12805    897
Name: trip_duration, Length: 4000, dtype: int64


C:\Users\DNS\AppData\Local\Temp\ipykernel_15780\4163948044.py:28: RuntimeWarning: overflow encountered in square
  return sum((y - np.dot(X, w)) ** 2) / X.shape[0]
C:\Users\DNS\AppData\Local\Temp\ipykernel_15780\4163948044.py:19: RuntimeWarning: invalid value encountered in scalar subtract
  dist = np.abs(loss - self.fun_error(X, y, w))


TypeError: unsupported operand type(s) for /: 'float' and 'tuple'

In [109]:
data.head(5)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2.0,1.452362e+09,1.452363e+09,1,-73.967682,40.792725,-73.990227,40.757504,588
1,1.0,1.459663e+09,1.459663e+09,1,-74.014114,40.712418,-74.002510,40.719894,282
2,1.0,1.455025e+09,1.455026e+09,1,-73.984634,40.768501,-73.972954,40.785343,667
3,1.0,1.463120e+09,1.463120e+09,1,-73.953064,40.768284,-73.953651,40.777622,314
4,1.0,1.455186e+09,1.455187e+09,1,-73.962204,40.778770,-73.982491,40.764336,892


In [135]:
X = data.iloc[:, :-1].values#data.drop(["trip_duration"], axis=1)
y = data.iloc[:, 8].values#data["trip_duration"]

In [108]:
print(y.values)

[ 588  282  667 ... 1810  480  458]


In [204]:
y_pred = lr.predict(X_test)
print(y_pred.shape, y_test.shape)
print(y_pred[3])
print(y_test[3])
print('MAE: ', lr.MAE_(y_test, y_pred))
print('MSE: ', lr.MSE_(y_test, y_pred))
print('RMAE: ', lr.RMSE_(y_test, y_pred))
print('MApE: ', lr.MAPE_(y_test, y_pred))
print('R_2: ', lr.R_2(y_test, y_pred))

AttributeError: 'NoneType' object has no attribute 'predict'

In [166]:
y_pred = lr.predict(X)
print(y_pred.shape, y_test.shape)
print(y_pred[3])
print(y[3])

(20000,) (4000,)
-1.9096855098038378e+162
314


In [ ]:
#L1 сам.
class Ridge_:
    
    Alpha = 0.5
    
    def __init__(self, Alpha):
        self.Alpha = Alpha;
        
    def fit(self, X, Y):

        self.m, self.n = X.shape
        self.W = np.zeros(self.n)

        self.X = X
        self.Y = Y

    
    def train(self):
        Y_predict = self.predict(self.X)
        
        dW = (-(2 * (self.X.T).dot(self.Y - Y_predict)) + 0.2) / self.m
        db = -2 * np.sum(self.Y - Y_predict) / self.m
        
        self.W = self.W - Alpha*dW
        return self

    def predict(self, X):
        return X.dot(self.W)+self.b    
        
        
    def fit(X, y):
        
    def predict(X_test):
        return np.dot(X, self.w)

In [167]:
#На память. Бесполезно.
def MAE(X, y):
    #X=X.as_matrix()
    w=np.linalg.pinv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
    s=0
    m=len(X.axes[0])
    n=len(X.axes[1])
    for i in range(m):
        y_=0
        for j in range(n):
            y_+=w[j]*X.iat[i,j]
        s+=abs(y.iat[i]-y_)
    s1=s/m
    return(s1)
def MSE(X, y):
    #X=X.as_matrix()
    w=np.linalg.pinv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
    s=0
    m=len(X.axes[0])
    n=len(X.axes[1])
    for i in range(m):
        y_=0
        for j in range(n):
            y_+=w[j]*X.iat[i,j]
        s+=(y.iat[i]-y_)**2
    s1=s/m
    return(s1)
def RMSE(X, y):
    #X=X.as_matrix()
    w=np.linalg.pinv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
    s=0
    m=len(X.axes[0])
    n=len(X.axes[1])
    for i in range(m):
        y_=0
        for j in range(n):
            y_+=w[j]*X.iat[i,j]
        s+=(y.iat[i]-y_)**2
    s1=sqrt(s/m)
    return(s1)
def MAPE(X, y):
    #X=X.as_matrix()
    w=np.linalg.pinv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
    s=0
    m=len(X.axes[0])
    n=len(X.axes[1])
    for i in range(m):
        y_=0
        for j in range(n):
            y_+=w[j]*X.iat[i,j]
        s+=abs((y.iat[i]-y_)/y.iat[i])
    s1=s/m
    return(s1)
def R_2(X, y):
    #X=X.as_matrix()
    w=np.linalg.pinv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
    s=0
    m=len(X.axes[0])
    for i in range(m):
        y_=y.mean()
        s+=(y.iat[i]-y_)**2
    s1=s/m
    return(1-(MSE(X,y)/s1))

In [172]:
print('MAE: ', MAE(X_train, y_train))
print('MSE: ', MSE(X_train, y_train))
print('RMAE: ', RMSE(X_train, y_train))
print('MApE: ', MAPE(X_train, y_train))
print('R_2: ', R_2(X_train, y_train))

print('MAE: ', MAE(X_test, y_test))
print('MSE: ', MSE(X_test, y_test))
print('RMAE: ', RMSE(X_test, y_test))
print('MApE: ', MAPE(X_test, y_test))
print('R_2: ', R_2(X_test, y_test))


MAE:  590.2542204070038
MSE:  8323202.462563874
RMAE:  2884.996094029223
MApE:  1.5424568755059902
R_2:  0.0013341793824208636
MAE:  661.7032032607909
MSE:  12457237.688627644
RMAE:  3529.481220891768
MApE:  1.674886310303826
R_2:  0.0003117740852730533
